In [1]:
import numpy as np
import pandas as pd
import requests
import csv
import ssl
from urllib.request import urlopen
from urllib.parse import quote_plus

In [2]:
pd.set_option('display.max_columns', None)

# Skirt 리뷰 전처리

In [118]:
df_skirt = pd.read_csv('./musinsa_review_skirt.csv')

In [119]:
df_skirt.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021.10.14,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021.08.22,2029080,0,1,159cm,60kg,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021.08.12,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021.08.06,1911322,0,1,158cm,54kg,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021.07.07,1911322,0,1,159cm,44kg,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0


In [120]:
df_skirt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12391 entries, 0 to 12390
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   userName       12375 non-null  object
 1   date           12391 non-null  object
 2   goodsNo        12391 non-null  int64 
 3   userSexMen     12391 non-null  int64 
 4   userSexWomen   12391 non-null  int64 
 5   userHeight     12391 non-null  object
 6   userWeight     12391 non-null  object
 7   goodsSize      12391 non-null  object
 8   reviewContent  12391 non-null  object
 9   reviewImg      3049 non-null   object
 10  reviewStyle    12391 non-null  int64 
 11  size           12391 non-null  int64 
 12  bright         12391 non-null  int64 
 13  color          12391 non-null  int64 
 14  thickness      12391 non-null  int64 
 15  weightness     12391 non-null  int64 
 16  touch          12391 non-null  int64 
 17  helpNo         12391 non-null  int64 
 18  styleLikeNo    12391 non-n

In [121]:
df_skirt.shape

(12391, 19)

### 탈퇴 인원 제거

In [122]:
df_skirt.dropna(subset="userName", axis=0, inplace=True)
df_skirt = df_skirt.reset_index(drop=True)

In [123]:
df_skirt.shape

(12375, 19)

### Date를 년, 월, 일로 등분

In [124]:
df_skirt.dropna(subset="date", axis=0, inplace=True)

In [125]:
df_skirt.shape

(12375, 19)

In [126]:
df_skirt.insert(2, "dateDay", 0, True)
df_skirt.insert(2, "dateMonth", 0, True)
df_skirt.insert(2, "dateYear", 0, True)

In [127]:
df_skirt.columns

Index(['userName', 'date', 'dateYear', 'dateMonth', 'dateDay', 'goodsNo',
       'userSexMen', 'userSexWomen', 'userHeight', 'userWeight', 'goodsSize',
       'reviewContent', 'reviewImg', 'reviewStyle', 'size', 'bright', 'color',
       'thickness', 'weightness', 'touch', 'helpNo', 'styleLikeNo'],
      dtype='object')

In [128]:
df_skirt['dateYear'] = df_skirt.apply(lambda x: x['date'].split('.')[0], axis=1)
df_skirt['dateMonth'] = df_skirt.apply(lambda x: x['date'].split('.')[1], axis=1)
df_skirt['dateDay'] = df_skirt.apply(lambda x: x['date'].split('.')[2], axis=1)

In [129]:
df_skirt.head()

,userName,date,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021.10.14,2021,10,14,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021.08.22,2021,08,22,2029080,0,1,159cm,60kg,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021.08.12,2021,08,12,2029080,0,1,158cm,48kg,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021.08.06,2021,08,06,1911322,0,1,158cm,54kg,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021.07.07,2021,07,07,1911322,0,1,159cm,44kg,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0


In [130]:
df_skirt = df_skirt.drop('date', axis=1)

In [131]:
df_skirt.dateYear = df_skirt.dateYear.astype('int')
df_skirt.dateMonth = df_skirt.dateMonth.astype('int')
df_skirt.dateDay = df_skirt.dateDay.astype('int')

### goodsNo

In [132]:
df_skirt.dropna(subset="goodsNo", axis=0, inplace=True)

In [133]:
df_skirt.shape

(12375, 21)

### 성별

In [134]:
df_skirt[df_skirt.userSexMen == 1].shape

(192, 21)

In [135]:
df_skirt[df_skirt.userSexWomen==1].shape

(12183, 21)

In [136]:
192+12183

12375

### 키와 몸무게

In [137]:
df_skirt.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [138]:
df_skirt.shape

(12375, 21)

In [139]:
df_skirt.userHeight = df_skirt.userHeight.apply(lambda x: x.replace("cm", ""))
df_skirt.userHeight = df_skirt.userHeight.astype('int64')

In [140]:
df_skirt.userWeight = df_skirt.userWeight.apply(lambda x: x.replace("kg", ""))
df_skirt.userWeight = df_skirt.userWeight.astype('int64')

### 이미지

In [141]:
df_skirt.reviewStyle.value_counts()

0    9328
1    3047
Name: reviewStyle, dtype: int64

### 사이즈

In [142]:
sub = df_skirt[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset="goodsNo", inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', sub.index[:], True)
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))

In [143]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [144]:
for i in range(2654):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
0,2029080,KHAKI/FREE,2029080,0
1,1911322,M,1911322,1
2,1911322,S,1911322,2
3,1625877,FREE,1625877,3
4,1586158,L(28-30),1586158,4
...,...,...,...,...
2649,2117256,S,2117256,2649
2650,2117256,M,2117256,2650
2651,2117256,L,2117256,2651
2652,2340070,M,2340070,2652


In [145]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2654 entries, 0 to 2653
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   goodsNo     2654 non-null   object
 1   goodsSize   2654 non-null   object
 2   newGoodsNo  2654 non-null   object
 3   index       2654 non-null   object
dtypes: object(4)
memory usage: 83.1+ KB


In [146]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub

,goodsNo,goodsSize,newGoodsNo,index
0,2029080,KHAKI/FREE,20290800,0
1,1911322,M,19113221,1
2,1911322,S,19113222,2
3,1625877,FREE,16258773,3
4,1586158,L(28-30),15861584,4
...,...,...,...,...
2649,2117256,S,21172562649,2649
2650,2117256,M,21172562650,2650
2651,2117256,L,21172562651,2651
2652,2340070,M,23400702652,2652


In [147]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

In [148]:
df_skirt

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021,10,14,2029080,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021,8,22,2029080,0,1,159,60,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021,8,12,2029080,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021,8,6,1911322,0,1,158,54,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021,7,7,1911322,0,1,159,44,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12370,뉴비_54bed8fe,2022,3,6,2340070,0,1,160,48,M,"['보통 여기 치마는 m 사면 무난하게 맞았는데', <br/>, '얘는 좀 허리가 ...",NaN,0,1,2,2,0,0,0,0,0
12371,얼래프,2022,3,3,2340070,0,1,163,47,S,['패턴은 특이하고 이쁜데 몸매 딱 잡아주는 슬림핏은 아니고 살짝 어벙벙해요 ㅠ'],NaN,0,1,2,2,0,0,0,0,0
12372,교촌한입만,2022,2,25,2340070,0,1,160,50,S,"['엉덩이 있는 편인데 S 잘맞아요', <br/>, '허리는 25-26 정도인데 적...",NaN,0,2,1,1,0,0,0,0,0
12373,야옹웅,2022,2,18,2340070,0,1,158,50,M,['실물로 보는게 더 이쁜거 같아요 진짜 이쁩니다 대박'],NaN,0,2,2,1,0,0,0,0,0


#### month 당 사는 옷 수

In [149]:
df_skirt.insert(5, 'month12', 0, True)
df_skirt.insert(5, 'month11', 0, True)
df_skirt.insert(5, 'month10', 0, True)
df_skirt.insert(5, 'month9', 0, True)
df_skirt.insert(5, 'month8', 0, True)
df_skirt.insert(5, 'month7', 0, True)
df_skirt.insert(5, 'month6', 0, True)
df_skirt.insert(5, 'month5', 0, True)
df_skirt.insert(5, 'month4', 0, True)
df_skirt.insert(5, 'month3', 0, True)
df_skirt.insert(5, 'month2', 0, True)
df_skirt.insert(5, 'month1', 0, True)

In [150]:
df_skirt.month1 = df_skirt.apply(lambda x: 1 if x.dateMonth==1 else x.month1, axis=1)
df_skirt.month2 = df_skirt.apply(lambda x: 1 if x.dateMonth==2 else x.month2, axis=1)
df_skirt.month3 = df_skirt.apply(lambda x: 1 if x.dateMonth==3 else x.month3, axis=1)
df_skirt.month4 = df_skirt.apply(lambda x: 1 if x.dateMonth==4 else x.month4, axis=1)
df_skirt.month5 = df_skirt.apply(lambda x: 1 if x.dateMonth==5 else x.month5, axis=1)
df_skirt.month6 = df_skirt.apply(lambda x: 1 if x.dateMonth==6 else x.month6, axis=1)
df_skirt.month7 = df_skirt.apply(lambda x: 1 if x.dateMonth==7 else x.month7, axis=1)
df_skirt.month8 = df_skirt.apply(lambda x: 1 if x.dateMonth==8 else x.month8, axis=1)
df_skirt.month9 = df_skirt.apply(lambda x: 1 if x.dateMonth==9 else x.month9, axis=1)
df_skirt.month10 = df_skirt.apply(lambda x: 1 if x.dateMonth==10 else x.month10, axis=1)
df_skirt.month11 = df_skirt.apply(lambda x: 1 if x.dateMonth==11 else x.month11, axis=1)
df_skirt.month12 = df_skirt.apply(lambda x: 1 if x.dateMonth==12 else x.month12, axis=1)

In [151]:
d = df_skirt.groupby(['goodsNo', 'goodsSize']).sum()[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]
d

,,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
goodsNo,goodsSize,,,,,,,,,,,,
387628,FREE/PINK,0,0,0,0,0,0,0,0,1,0,0,0
387631,FREE/BLUE,0,0,0,0,0,0,0,1,0,0,0,0
387632,FREE/NAVY,0,0,1,0,0,0,0,0,0,0,0,0
407626,FREE,0,0,1,0,0,0,0,0,0,0,0,0
463655,FREE,0,0,2,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2389731,FREE,0,0,1,0,0,0,0,0,0,0,0,0
2391576,S,0,0,1,0,0,0,0,0,0,0,0,0
2391598,S,0,0,1,0,0,0,0,0,0,0,0,0


#### 키, 몸무게 평균 구하기

In [152]:
c = df_skirt.groupby(['goodsNo', 'goodsSize'], as_index=False).mean()[['goodsNo', 'goodsSize', 'userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]
c

,goodsNo,goodsSize,userHeight,userWeight,userSexMen,userSexWomen
0,387628,FREE/PINK,160.0,47.0,0.0,1.0
1,387631,FREE/BLUE,158.0,46.0,0.0,1.0
2,387632,FREE/NAVY,163.0,50.0,0.0,1.0
3,407626,FREE,161.0,54.0,0.0,1.0
4,463655,FREE,150.0,50.0,0.0,1.0
...,...,...,...,...,...,...
2649,2389731,FREE,165.0,55.0,0.0,1.0
2650,2391576,S,161.0,46.0,0.0,1.0
2651,2391598,S,158.0,46.0,0.0,1.0
2652,2394978,M,160.0,55.0,0.0,1.0


In [153]:
sub.sort_values(['goodsNo', 'goodsSize'], inplace=True)
c.sort_values(['goodsNo', 'goodsSize'], inplace=True)
d.sort_values(['goodsNo', 'goodsSize'], inplace=True)
sub

,goodsNo,goodsSize,newGoodsNo,index
1116,387628,FREE/PINK,38762801116,1116
1604,387631,FREE/BLUE,38763101604,1604
1044,387632,FREE/NAVY,38763201044,1044
740,407626,FREE,4076260740,740
465,463655,FREE,4636550465,465
...,...,...,...,...
2219,2389731,FREE,23897312219,2219
2318,2391576,S,23915762318,2318
1607,2391598,S,23915981607,1607
371,2394978,M,2394978371,371


In [154]:
d.index = range(len(d))
d

,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,2,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2649,0,0,1,0,0,0,0,0,0,0,0,0
2650,0,0,1,0,0,0,0,0,0,0,0,0
2651,0,0,1,0,0,0,0,0,0,0,0,0
2652,0,0,1,0,0,0,0,0,0,0,0,0


In [155]:
sub = pd.concat([sub, c[['userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]], axis=1)
sub = pd.concat([sub, d[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]], axis=1)
sub.index = range(len(sub))
sub

,goodsNo,goodsSize,newGoodsNo,index,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,387628,FREE/PINK,38762801116,1116,162.702128,51.744681,0.042553,0.957447,9,10,0,1,0,0,0,0,1,3,11,12
1,387631,FREE/BLUE,38763101604,1604,165.000000,50.000000,0.000000,1.000000,0,0,2,0,0,0,0,0,0,0,0,0
2,387632,FREE/NAVY,38763201044,1044,163.000000,50.000000,0.000000,1.000000,2,0,0,0,0,0,0,0,0,0,0,0
3,407626,FREE,4076260740,740,165.000000,52.000000,0.000000,1.000000,0,1,0,0,0,0,0,0,0,0,0,0
4,463655,FREE,4636550465,465,161.733333,53.933333,0.066667,0.933333,0,2,0,0,1,3,2,1,1,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649,2389731,FREE,23897312219,2219,155.000000,50.000000,0.000000,1.000000,0,0,0,0,0,0,0,0,0,1,0,0
2650,2391576,S,23915762318,2318,159.625000,47.750000,0.125000,0.875000,4,3,1,0,0,0,0,0,0,0,0,0
2651,2391598,S,23915981607,1607,167.294118,65.647059,0.000000,1.000000,0,0,0,0,1,4,0,0,4,6,1,1
2652,2394978,M,2394978371,371,164.000000,52.250000,0.000000,1.000000,0,0,0,0,0,0,0,0,0,0,3,1


In [156]:
df_skirt = pd.merge(left=df_skirt, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_skirt.head()

,userName,dateYear,dateMonth,dateDay,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,k_hnnj,2021,10,14,2029080,0,0,0,0,0,0,0,0,0,1,0,0,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0,20290800
1,ꉂꉂᵔᗜᵔ,2021,8,22,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,159,60,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0,20290800
2,k_hnnj,2021,8,12,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0,20290800
3,미역이233,2021,8,6,1911322,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,54,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0,19113221
4,블리츠크랭크!,2021,7,7,1911322,0,0,0,0,0,0,1,0,0,0,0,0,0,1,159,44,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0,19113222


In [159]:
len(sub['goodsNo'].unique())

1800

In [160]:
sub.shape

(2654, 20)

In [188]:
sub.to_csv('size_skirt.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

### Contents

In [189]:
print(df_skirt.reviewContent[0])
print(df_skirt.reviewContent[1].split(","))

['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용']
["['생각보다 짧은 치마'", ' <br/>', " '제 체격에는 좀 작게 느껴질만한 사이즈'", ' <br/>', " '디자인은 되게 예쁨']"]


In [190]:
cnt = 0
result = []
for i in df_skirt.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

2


In [191]:
df_skirt[df_skirt.reviewContent=='[\'데님 색이 너무 맘에 들어서 구매했습니다!\', <br/>, \'키가 작아서 길이가 너무 길면 어떡하나 싶었는데 길이도 잘 맞고 허리도 너무 잘 맞았습니다>_\', \'!</div\', \'\\n\', <div class="review-evaluation">\n<ul class="review-evaluation__list">\n<li class="review-evaluation__item">사이즈 <span>보통이에요</span></li>\n<li class="review-evaluation__item">밝기 <span>보통이에요</span></li>\n<li class="review-evaluation__item">색감 <span>보통이에요</span></li>\n</ul>\n</div>, \'\\n\', \' 이미지 \', \'\\n\', <div class="review-content-photo">\n<div class="review-content-photo__wrap">\n<ul class="review-content-photo__list">\n<li class="review-content-photo__item" data-img-index="0" style="background-image:url(\'//image.msscdn.net/data/estimate/1618698_0/gallery_607d08ce95b27.jpg.list\');">\n<img alt="밀리언코르(MILLIONCOR) [DANA 8130]커버밴드 A라인 데님 스커트(블랙그레이) 후기" src="//image.msscdn.net/data/estimate/1618698_0/gallery_607d08ce95b27.jpg.view"/>\n</li>\n</ul>\n</div>\n</div>, \'\\n\', \' 이모티콘 \', \'\\n\', \' SNS 공유시 필요한 데이터 \', \'\\n\', <span class="p_name" style="display: none;">[DANA 8130]커버밴드 A라인 데님 스커트(블랙그레이)</span>, \'\\n\']']

,userName,dateYear,dateMonth,dateDay,newClothNo,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
1184,여신예리,2021,4,19,1618698367,1618698,0,0,0,1,0,0,0,0,0,0,0,0,0,1,152,53,M,"['데님 색이 너무 맘에 들어서 구매했습니다!', <br/>, '키가 작아서 길이가...",https://image.msscdn.net/data/estimate/1618698...,1,2,2,2,0,0,0,0,0


In [192]:
df_skirt[df_skirt.reviewContent=='[\'허리 밴딩이 넓어서 좋고 싸이즈도 은근 생각보다 커서 S 사이즈 사게 잘한거 같애요!! 예뻐요~>\', \'!</div\', \'\\n\', <div class="review-evaluation">\n<ul class="review-evaluation__list">\n<li class="review-evaluation__item">사이즈 <span>보통이에요</span></li>\n<li class="review-evaluation__item">밝기 <span>보통이에요</span></li>\n<li class="review-evaluation__item">색감 <span>보통이에요</span></li>\n</ul>\n</div>, \'\\n\', \' 이미지 \', \'\\n\', \' 이모티콘 \', \'\\n\', \' SNS 공유시 필요한 데이터 \', \'\\n\', <span class="p_name" style="display: none;">Inside-Out Kangaroo Skirt Melange-Reverse</span>, \'\\n\']']

,userName,dateYear,dateMonth,dateDay,newClothNo,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
11323,써드3,2021,7,23,17887982306,1788798,0,0,0,0,0,0,1,0,0,0,0,0,0,1,163,50,S,['허리 밴딩이 넓어서 좋고 싸이즈도 은근 생각보다 커서 S 사이즈 사게 잘한거 같...,NaN,0,2,2,2,0,0,0,0,0


In [193]:
df_skirt.loc[1186, 'reviewContent'] = result[0].split('</div')[0][:]
df_skirt.loc[11338, 'reviewContent'] = result[1].split('</div')[0][:]

In [194]:
print(df_skirt.loc[1186].reviewContent)
print(df_skirt.loc[11338].reviewContent)

['데님 색이 너무 맘에 들어서 구매했습니다!', <br/>, '키가 작아서 길이가 너무 길면 어떡하나 싶었는데 길이도 잘 맞고 허리도 너무 잘 맞았습니다>_', '!
['허리 밴딩이 넓어서 좋고 싸이즈도 은근 생각보다 커서 S 사이즈 사게 잘한거 같애요!! 예뻐요~>', '!


In [195]:
df_skirt.reviewContent = df_skirt.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("'", "").replace(", <br/>", "").split(","))

In [196]:
print(df_skirt.reviewContent[0])
print(df_skirt.reviewContent[1])

['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용']
['생각보다 짧은 치마', '제 체격에는 좀 작게 느껴질만한 사이즈', '디자인은 되게 예쁨']


### 그 외

In [197]:
df_skirt['size'].value_counts()

2    9747
1    1872
3     756
Name: size, dtype: int64

In [198]:
df_skirt.bright.value_counts()

2    10196
3     1211
1      968
Name: bright, dtype: int64

In [199]:
df_skirt.color.value_counts()

2    8745
1    3468
3     162
Name: color, dtype: int64

In [200]:
df_skirt.thickness.value_counts()

0    12372
2        2
1        1
Name: thickness, dtype: int64

In [201]:
df_skirt.weightness.value_counts()

0    12375
Name: weightness, dtype: int64

In [202]:
df_skirt.touch.value_counts()

0    12375
Name: touch, dtype: int64

### helpNo, styleLikeNo는 모두 0이다. 제거 가능성 높아짐

In [203]:
df_skirt.head()

,userName,dateYear,dateMonth,dateDay,newClothNo,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,k_hnnj,2021,10,14,20290800,2029080,0,0,0,0,0,0,0,0,0,1,0,0,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0
1,ꉂꉂᵔᗜᵔ,2021,8,22,20290800,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,159,60,KHAKI/FREE,"[생각보다 짧은 치마, 제 체격에는 좀 작게 느껴질만한 사이즈, 디자인은 되게 예쁨]",NaN,0,3,2,2,0,0,0,0,0
2,k_hnnj,2021,8,12,20290800,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,48,KHAKI/FREE,[핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용],NaN,0,2,2,2,0,0,0,0,0
3,미역이233,2021,8,6,19113221,1911322,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,54,M,"[주름이 많이 가긴 하는데 스판도 좋고 이쁩니다, 제 키에는 좀 길어서 무릎까지 오...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0
4,블리츠크랭크!,2021,7,7,19113222,1911322,0,0,0,0,0,0,1,0,0,0,0,0,0,1,159,44,S,[리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나고...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0


## User 데이터 생성 (unique한 유저 정보로 수정)

user_skirt = df_skirt[['userName', 'userSexMen', 'userSexWomen']]
user_skirt = user_skirt.drop_duplicates(subset=['userName'])
user_skirt.sort_values('userName', inplace=True)
user_skirt.index = range(len(user_skirt))

mo = df_skirt.groupby('userName').sum()[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]
mo.sort_values('userName', inplace=True)
mo.index = range(len(mo))

hw = df_skirt.groupby('userName').mean()[['userHeight', 'userWeight']]
hw.sort_values('userName', inplace=True)
hw.index = range(len(hw))

user_skirt = pd.concat([user_skirt, hw, mo], axis=1)
user_skirt

user_skirt[user_skirt.userName=="사이먼0112"]

user_skirt[user_skirt.userName=='k_hnnj'].index

## Transaction 데이터 생성(전체 리뷰인 경우로 수정)

transaction_data = df_skirt[['userName', 'newClothNo']]
transaction_data.insert(0, 'userNo', 0, True)

transaction_data.userNo = transaction_data.apply(lambda x: user_skirt[user_skirt.userName==x.userName].index[0], axis=1)

transaction_data = transaction_data.drop('userName', axis=1)
transaction_data

# Onepiece 리뷰 전처리

In [49]:
df_onepiece = pd.read_csv('./musinsa_review_onepiece.csv')

In [50]:
df_onepiece.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,odddr,2021.05.29,1933622,0,1,163cm,53kg,none,['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 ...,https://image.msscdn.net/data/estimate/1933622...,1,1,2,2,0,0,0,0,0
1,odddr,2021.05.29,1933622,0,1,163cm,53kg,none,"['여름에 편하게 입기 좋을것 같아요!', <br/>, '생각보다 많이 파여서 조심...",NaN,0,1,2,2,0,0,0,0,0
2,뉴비_a7b53dc7,2022.02.25,2297184,0,1,160cm,55kg,M,"['길이감 좋고 스타일 꾸안꾸느낌', <br/>, '빨리 날 좋아지면 입고싶음', ...",https://image.msscdn.net/data/estimate/2297184...,1,2,2,2,0,0,0,0,0
3,뉴비_a7b53dc7,2022.02.25,2297184,0,1,160cm,55kg,M,"['여유있게 입으려고 M사이즈 구매', <br/>, '나그랑이라 어깨 안넓어보이고 ...",NaN,0,2,2,2,0,0,0,0,0
4,YnSUGA,2022.02.22,2297184,0,1,152cm,58kg,S,['봄 신상이라 그런가 확실히 얇아요. 키가 작은 편이라 기장 수선해서 입어야 할 ...,NaN,0,2,2,2,0,0,0,0,0


In [51]:
df_onepiece.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8461 entries, 0 to 8460
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   userName       8448 non-null   object
 1   date           8461 non-null   object
 2   goodsNo        8461 non-null   int64 
 3   userSexMen     8461 non-null   int64 
 4   userSexWomen   8461 non-null   int64 
 5   userHeight     8461 non-null   object
 6   userWeight     8461 non-null   object
 7   goodsSize      8461 non-null   object
 8   reviewContent  8461 non-null   object
 9   reviewImg      2117 non-null   object
 10  reviewStyle    8461 non-null   int64 
 11  size           8461 non-null   int64 
 12  bright         8461 non-null   int64 
 13  color          8461 non-null   int64 
 14  thickness      8461 non-null   int64 
 15  weightness     8461 non-null   int64 
 16  touch          8461 non-null   int64 
 17  helpNo         8461 non-null   int64 
 18  styleLikeNo    8461 non-null

In [52]:
df_onepiece.shape

(8461, 19)

### 탈퇴 인원 제거

In [53]:
df_onepiece.dropna(subset="userName", axis=0, inplace=True)
df_onepiece = df_onepiece.reset_index(drop=True)

In [54]:
df_onepiece.shape

(8448, 19)

### Date를 년, 월, 일로 등분

In [55]:
df_onepiece.dropna(subset="date", axis=0, inplace=True)

In [56]:
df_onepiece.shape

(8448, 19)

In [57]:
df_onepiece.insert(2, "dateDay", 0, True)
df_onepiece.insert(2, "dateMonth", 0, True)
df_onepiece.insert(2, "dateYear", 0, True)

In [58]:
df_onepiece.columns

Index(['userName', 'date', 'dateYear', 'dateMonth', 'dateDay', 'goodsNo',
       'userSexMen', 'userSexWomen', 'userHeight', 'userWeight', 'goodsSize',
       'reviewContent', 'reviewImg', 'reviewStyle', 'size', 'bright', 'color',
       'thickness', 'weightness', 'touch', 'helpNo', 'styleLikeNo'],
      dtype='object')

In [59]:
df_onepiece['dateYear'] = df_onepiece.apply(lambda x: x['date'].split('.')[0], axis=1)
df_onepiece['dateMonth'] = df_onepiece.apply(lambda x: x['date'].split('.')[1], axis=1)
df_onepiece['dateDay'] = df_onepiece.apply(lambda x: x['date'].split('.')[2], axis=1)

In [60]:
df_onepiece.head()

,userName,date,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,odddr,2021.05.29,2021,05,29,1933622,0,1,163cm,53kg,none,['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 ...,https://image.msscdn.net/data/estimate/1933622...,1,1,2,2,0,0,0,0,0
1,odddr,2021.05.29,2021,05,29,1933622,0,1,163cm,53kg,none,"['여름에 편하게 입기 좋을것 같아요!', <br/>, '생각보다 많이 파여서 조심...",NaN,0,1,2,2,0,0,0,0,0
2,뉴비_a7b53dc7,2022.02.25,2022,02,25,2297184,0,1,160cm,55kg,M,"['길이감 좋고 스타일 꾸안꾸느낌', <br/>, '빨리 날 좋아지면 입고싶음', ...",https://image.msscdn.net/data/estimate/2297184...,1,2,2,2,0,0,0,0,0
3,뉴비_a7b53dc7,2022.02.25,2022,02,25,2297184,0,1,160cm,55kg,M,"['여유있게 입으려고 M사이즈 구매', <br/>, '나그랑이라 어깨 안넓어보이고 ...",NaN,0,2,2,2,0,0,0,0,0
4,YnSUGA,2022.02.22,2022,02,22,2297184,0,1,152cm,58kg,S,['봄 신상이라 그런가 확실히 얇아요. 키가 작은 편이라 기장 수선해서 입어야 할 ...,NaN,0,2,2,2,0,0,0,0,0


In [61]:
df_onepiece = df_onepiece.drop('date', axis=1)

In [62]:
df_onepiece.dateYear = df_onepiece.dateYear.astype('int')
df_onepiece.dateMonth = df_onepiece.dateMonth.astype('int')
df_onepiece.dateDay = df_onepiece.dateDay.astype('int')

### goodsNo

In [63]:
df_onepiece.dropna(subset="goodsNo", axis=0, inplace=True)

In [64]:
df_onepiece.shape

(8448, 21)

### 성별

In [65]:
df_onepiece[df_onepiece.userSexMen == 1].shape

(144, 21)

In [66]:
df_onepiece[df_onepiece.userSexWomen==1].shape

(8304, 21)

In [67]:
144+8304

8448

### 키와 몸무게

In [68]:
df_onepiece.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [69]:
df_onepiece.shape

(8448, 21)

In [70]:
df_onepiece.userHeight = df_onepiece.userHeight.apply(lambda x: x.replace("cm", ""))
df_onepiece.userHeight = df_onepiece.userHeight.astype('int64')

In [71]:
df_onepiece.userWeight = df_onepiece.userWeight.apply(lambda x: x.replace("kg", ""))
df_onepiece.userWeight = df_onepiece.userWeight.astype('int64')

### 이미지

In [72]:
df_onepiece.reviewStyle.value_counts()

0    6332
1    2116
Name: reviewStyle, dtype: int64

In [73]:
2654+2047

4701

### 사이즈

In [74]:
sub = df_onepiece[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset="goodsNo", inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', 0, True)
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))
sub['index'] = range(2654, 2654+len(sub))
sub.index = range(2654, 2654+len(sub))
sub

,goodsNo,goodsSize,newGoodsNo,index
2654,1933622,none,1933622,2654
2655,2297184,M,2297184,2655
2656,2297184,S,2297184,2656
2657,2297184,XS,2297184,2657
2658,1771393,MINT/FREE,1771393,2658
...,...,...,...,...
4696,1782233,M,1782233,4696
4697,1790851,M,1790851,4697
4698,1440679,NONE,1440679,4698
4699,1941295,PINK/FREE,1941295,4699


In [75]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [76]:
len(sub)

2047

In [77]:
for i in range(2654, len(sub)+2654):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
2654,1933622,none,1933622,2654
2655,2297184,M,2297184,2655
2656,2297184,S,2297184,2656
2657,2297184,XS,2297184,2657
2658,1771393,MINT/FREE,1771393,2658
...,...,...,...,...
4696,1782233,M,1782233,4696
4697,1790851,M,1790851,4697
4698,1440679,NONE,1440679,4698
4699,1941295,PINK/FREE,1941295,4699


In [78]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2047 entries, 2654 to 4700
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   goodsNo     2047 non-null   object
 1   goodsSize   2047 non-null   object
 2   newGoodsNo  2047 non-null   object
 3   index       2047 non-null   object
dtypes: object(4)
memory usage: 64.1+ KB


In [79]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub

,goodsNo,goodsSize,newGoodsNo,index
2654,1933622,none,19336222654,2654
2655,2297184,M,22971842655,2655
2656,2297184,S,22971842656,2656
2657,2297184,XS,22971842657,2657
2658,1771393,MINT/FREE,17713932658,2658
...,...,...,...,...
4696,1782233,M,17822334696,4696
4697,1790851,M,17908514697,4697
4698,1440679,NONE,14406794698,4698
4699,1941295,PINK/FREE,19412954699,4699


In [80]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

In [81]:
df_onepiece

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,odddr,2021,5,29,1933622,0,1,163,53,none,['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 ...,https://image.msscdn.net/data/estimate/1933622...,1,1,2,2,0,0,0,0,0
1,odddr,2021,5,29,1933622,0,1,163,53,none,"['여름에 편하게 입기 좋을것 같아요!', <br/>, '생각보다 많이 파여서 조심...",NaN,0,1,2,2,0,0,0,0,0
2,뉴비_a7b53dc7,2022,2,25,2297184,0,1,160,55,M,"['길이감 좋고 스타일 꾸안꾸느낌', <br/>, '빨리 날 좋아지면 입고싶음', ...",https://image.msscdn.net/data/estimate/2297184...,1,2,2,2,0,0,0,0,0
3,뉴비_a7b53dc7,2022,2,25,2297184,0,1,160,55,M,"['여유있게 입으려고 M사이즈 구매', <br/>, '나그랑이라 어깨 안넓어보이고 ...",NaN,0,2,2,2,0,0,0,0,0
4,YnSUGA,2022,2,22,2297184,0,1,152,58,S,['봄 신상이라 그런가 확실히 얇아요. 키가 작은 편이라 기장 수선해서 입어야 할 ...,NaN,0,2,2,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8443,yu.r,2020,9,4,1440679,0,1,163,52,NONE,['패턴이 너무 이뻐서 구매했는데 생각보다 치마길이는 짧아서 너무 아쉬웠어요 ㅠㅠ ...,NaN,0,3,2,2,0,0,0,0,0
8444,뚝서,2021,8,10,1941295,0,1,160,50,PINK/FREE,"['원피스는 통통하신 분들도 잘 들어갈 정도로 밴딩이 잘 되어 있어요', <br/>...",NaN,0,2,2,1,0,0,0,0,0
8445,뉴비_ed629198,2021,9,4,1464994,0,1,160,48,FREE,['정말정말 만족하는 원피스 입니다 생각한것 보다 훨씬 예쁘고 봄여름가을 상관없이 ...,https://image.msscdn.net/data/estimate/1464994...,1,2,2,2,0,0,0,0,0
8446,소영파크,2022,2,9,1464994,0,1,170,60,FREE,['여리여리해보이고 샤랄라하고 사이즈 프리지만 길이감도 적당'],NaN,0,2,2,2,0,0,0,0,0


#### month 당 사는 옷 수

In [82]:
df_skirt = pd.merge(left=df_skirt, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_skirt.head()

len(sub['goodsNo'].unique())

sub.shape

sub.to_csv('size_skirt.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')df_onepiece.insert(5, 'month12', 0, True)
df_onepiece.insert(5, 'month11', 0, True)
df_onepiece.insert(5, 'month10', 0, True)
df_onepiece.insert(5, 'month9', 0, True)
df_onepiece.insert(5, 'month8', 0, True)
df_onepiece.insert(5, 'month7', 0, True)
df_onepiece.insert(5, 'month6', 0, True)
df_onepiece.insert(5, 'month5', 0, True)
df_onepiece.insert(5, 'month4', 0, True)
df_onepiece.insert(5, 'month3', 0, True)
df_onepiece.insert(5, 'month2', 0, True)
df_onepiece.insert(5, 'month1', 0, True)

In [83]:
df_onepiece.month1 = df_onepiece.apply(lambda x: 1 if x.dateMonth==1 else x.month1, axis=1)
df_onepiece.month2 = df_onepiece.apply(lambda x: 1 if x.dateMonth==2 else x.month2, axis=1)
df_onepiece.month3 = df_onepiece.apply(lambda x: 1 if x.dateMonth==3 else x.month3, axis=1)
df_onepiece.month4 = df_onepiece.apply(lambda x: 1 if x.dateMonth==4 else x.month4, axis=1)
df_onepiece.month5 = df_onepiece.apply(lambda x: 1 if x.dateMonth==5 else x.month5, axis=1)
df_onepiece.month6 = df_onepiece.apply(lambda x: 1 if x.dateMonth==6 else x.month6, axis=1)
df_onepiece.month7 = df_onepiece.apply(lambda x: 1 if x.dateMonth==7 else x.month7, axis=1)
df_onepiece.month8 = df_onepiece.apply(lambda x: 1 if x.dateMonth==8 else x.month8, axis=1)
df_onepiece.month9 = df_onepiece.apply(lambda x: 1 if x.dateMonth==9 else x.month9, axis=1)
df_onepiece.month10 = df_onepiece.apply(lambda x: 1 if x.dateMonth==10 else x.month10, axis=1)
df_onepiece.month11 = df_onepiece.apply(lambda x: 1 if x.dateMonth==11 else x.month11, axis=1)
df_onepiece.month12 = df_onepiece.apply(lambda x: 1 if x.dateMonth==12 else x.month12, axis=1)

In [84]:
d = df_onepiece.groupby(['goodsNo', 'goodsSize']).sum()[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]
d

,,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
goodsNo,goodsSize,,,,,,,,,,,,
230853,none,0,0,0,0,0,0,1,0,0,0,0,0
387218,그레이헤더/S,0,0,0,0,0,0,0,1,0,0,0,0
435992,NONE,0,0,0,1,0,0,0,0,0,0,0,0
653344,none,0,0,0,0,0,0,0,0,0,0,1,0
677397,FREE,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386153,M,0,0,1,0,0,0,0,0,0,0,0,0
2396014,M,0,0,1,0,0,0,0,0,0,0,0,0
2396257,FREE,0,0,2,0,0,0,0,0,0,0,0,0


#### 키, 몸무게 평균 구하기

In [85]:
c = df_onepiece.groupby(['goodsNo', 'goodsSize'], as_index=False).mean()[['goodsNo', 'goodsSize', 'userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]
c

,goodsNo,goodsSize,userHeight,userWeight,userSexMen,userSexWomen
0,230853,none,155.0,45.0,0.0,1.0
1,387218,그레이헤더/S,161.0,43.0,0.0,1.0
2,435992,NONE,162.0,55.0,0.0,1.0
3,653344,none,155.0,55.0,0.0,1.0
4,677397,FREE,163.0,55.0,0.0,1.0
...,...,...,...,...,...,...
2042,2386153,M,171.0,58.0,0.0,1.0
2043,2396014,M,173.0,53.0,0.0,1.0
2044,2396257,FREE,158.0,48.0,0.0,1.0
2045,2396259,FREE,158.0,48.0,0.0,1.0


In [86]:
sub.sort_values(['goodsNo', 'goodsSize'], inplace=True)
c.sort_values(['goodsNo', 'goodsSize'], inplace=True)
d.sort_values(['goodsNo', 'goodsSize'], inplace=True)
sub.index = range(len(d))
sub

,goodsNo,goodsSize,newGoodsNo,index
0,230853,none,23085304022,4022
1,387218,그레이헤더/S,38721803234,3234
2,435992,NONE,43599203203,3203
3,653344,none,65334404183,4183
4,677397,FREE,67739703065,3065
...,...,...,...,...
2042,2386153,M,23861533521,3521
2043,2396014,M,23960143328,3328
2044,2396257,FREE,23962573722,3722
2045,2396259,FREE,23962593321,3321


In [87]:
d.index = range(len(d))
d

,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2042,0,0,1,0,0,0,0,0,0,0,0,0
2043,0,0,1,0,0,0,0,0,0,0,0,0
2044,0,0,2,0,0,0,0,0,0,0,0,0
2045,0,0,2,0,0,0,0,0,0,0,0,0


In [88]:
sub = pd.concat([sub, c[['userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]], axis=1)
sub = pd.concat([sub, d[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]], axis=1)
sub.index = range(len(sub))
sub

,goodsNo,goodsSize,newGoodsNo,index,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,230853,none,23085304022,4022,155.0,45.0,0.0,1.0,0,0,0,0,0,0,1,0,0,0,0,0
1,387218,그레이헤더/S,38721803234,3234,161.0,43.0,0.0,1.0,0,0,0,0,0,0,0,1,0,0,0,0
2,435992,NONE,43599203203,3203,162.0,55.0,0.0,1.0,0,0,0,1,0,0,0,0,0,0,0,0
3,653344,none,65334404183,4183,155.0,55.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,1,0
4,677397,FREE,67739703065,3065,163.0,55.0,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2042,2386153,M,23861533521,3521,171.0,58.0,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0
2043,2396014,M,23960143328,3328,173.0,53.0,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0
2044,2396257,FREE,23962573722,3722,158.0,48.0,0.0,1.0,0,0,2,0,0,0,0,0,0,0,0,0
2045,2396259,FREE,23962593321,3321,158.0,48.0,0.0,1.0,0,0,2,0,0,0,0,0,0,0,0,0


In [156]:
df_onepiece = pd.merge(left=df_onepiece, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_onepiece.head()

,userName,dateYear,dateMonth,dateDay,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,k_hnnj,2021,10,14,2029080,0,0,0,0,0,0,0,0,0,1,0,0,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],https://image.msscdn.net/data/estimate/2029080...,1,2,2,2,0,0,0,0,0,20290800
1,ꉂꉂᵔᗜᵔ,2021,8,22,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,159,60,KHAKI/FREE,"['생각보다 짧은 치마', <br/>, '제 체격에는 좀 작게 느껴질만한 사이즈',...",NaN,0,3,2,2,0,0,0,0,0,20290800
2,k_hnnj,2021,8,12,2029080,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,48,KHAKI/FREE,['핏 예뻐요 길이도 적당하구요 다만 허리가 좀 큰것 같아용'],NaN,0,2,2,2,0,0,0,0,0,20290800
3,미역이233,2021,8,6,1911322,0,0,0,0,0,0,0,1,0,0,0,0,0,1,158,54,M,"['주름이 많이 가긴 하는데 스판도 좋고 이쁩니다', <br/>, '제 키에는 좀 ...",https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0,19113221
4,블리츠크랭크!,2021,7,7,1911322,0,0,0,0,0,0,1,0,0,0,0,0,0,1,159,44,S,['리올그 다른치마는 허리 좀 컸는데 이건 잘 맞아요. 좀 구겨지긴 하지만 잘늘어나...,https://image.msscdn.net/data/estimate/1911322...,1,2,2,2,0,0,0,0,0,19113222


In [159]:
len(sub['goodsNo'].unique())

1800

In [160]:
sub.shape

(2654, 20)

In [188]:
sub.to_csv('size_onepiece.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

### Contents

In [92]:
print(df_onepiece.reviewContent[0])
print(df_onepiece.reviewContent[1].split(","))

['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 사이즈 구애받지 않고 다 입으실 수 있을거 같아요! 허리끈이 따로 있어서 다양하게 스타일링 할 수 있어 좋네요!']
["['여름에 편하게 입기 좋을것 같아요!'", ' <br/>', " '생각보다 많이 파여서 조심해야ㅎㅐ요ㅠㅠ'", ' <br/>', " '앞뒤가 언발란스한 기장때문에 심심하지 않아 좋네요!']"]


In [93]:
# span 태그가 들어가있는 리뷰내용 전처리

cnt = 0
result = []
for i in df_onepiece.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

0


In [94]:
df_onepiece.reviewContent = df_onepiece.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("'", "").replace(", <br/>", "").split(","))

In [95]:
print(df_onepiece.reviewContent[0])
print(df_onepiece.reviewContent[1])

['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 사이즈 구애받지 않고 다 입으실 수 있을거 같아요! 허리끈이 따로 있어서 다양하게 스타일링 할 수 있어 좋네요!']
['여름에 편하게 입기 좋을것 같아요!', '생각보다 많이 파여서 조심해야ㅎㅐ요ㅠㅠ', '앞뒤가 언발란스한 기장때문에 심심하지 않아 좋네요!']


### 그 외

In [96]:
df_onepiece['size'].value_counts()

2    6916
1    1103
3     428
0       1
Name: size, dtype: int64

In [97]:
df_onepiece.bright.value_counts()

2    6711
1     934
3     802
0       1
Name: bright, dtype: int64

In [98]:
df_onepiece.color.value_counts()

2    5667
1    2646
3     134
0       1
Name: color, dtype: int64

In [99]:
df_onepiece.thickness.value_counts()

0    8446
3       2
Name: thickness, dtype: int64

In [100]:
df_onepiece.weightness.value_counts()

0    8448
Name: weightness, dtype: int64

In [101]:
df_onepiece.touch.value_counts()

0    8448
Name: touch, dtype: int64

### helpNo, styleLikeNo는 모두 0이다. 제거 가능성 높아짐

In [102]:
df_onepiece.describe()

,dateYear,dateMonth,dateDay,newClothNo,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
count,8448.000000,8448.000000,8448.000000,8.448000e+03,8.448000e+03,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.0,8448.0,8448.0,8448.0
mean,2021.150331,6.627012,15.122514,1.867238e+10,1.813087e+06,0.079664,0.104167,0.081084,0.035985,0.059067,0.077770,0.109730,0.122633,0.089252,0.088778,0.072798,0.079072,0.017045,0.982955,162.436553,52.915601,0.250473,1.919863,1.984138,1.702415,0.000710,0.0,0.0,0.0,0.0
std,0.503225,3.419035,8.839225,6.333033e+09,3.012917e+05,0.270788,0.305495,0.272981,0.186263,0.235764,0.267825,0.312572,0.328034,0.285124,0.284440,0.259821,0.269867,0.129448,0.129448,5.925707,12.765438,0.433311,0.418687,0.453584,0.490935,0.046157,0.0,0.0,0.0,0.0
min,2020.000000,1.000000,1.000000,1.007885e+10,2.308530e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2021.000000,3.000000,7.000000,1.622447e+10,1.609158e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,159.000000,48.000000,0.000000,2.000000,2.000000,1.000000,0.000000,0.0,0.0,0.0,0.0
50%,2021.000000,7.000000,15.000000,1.888746e+10,1.877330e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,162.000000,52.000000,0.000000,2.000000,2.000000,2.000000,0.000000,0.0,0.0,0.0,0.0
75%,2021.000000,9.000000,23.000000,2.016787e+10,2.009663e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,166.000000,56.000000,1.000000,2.000000,2.000000,2.000000,0.000000,0.0,0.0,0.0,0.0
max,2022.000000,12.000000,31.000000,9.930520e+10,2.397414e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,200.000000,1000.000000,1.000000,3.000000,3.000000,3.000000,3.000000,0.0,0.0,0.0,0.0


# Pants 리뷰 전처리

In [3]:
df_pants1 = pd.read_csv('./musinsa_review_pants_1.csv')
df_pants2 = pd.read_csv('./musinsa_review_pants_2.csv')

In [4]:
print(df_pants1.shape, df_pants2.shape)

(558110, 19) (946857, 19)


In [5]:
df_pants = pd.concat([df_pants1, df_pants2], axis=0)
df_pants.shape

(1504967, 19)

In [6]:
df_pants

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020.12.23,1281045,1,0,173cm,69kg,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946852,마산동호랭이,2020.08.10,1168907,1,0,178cm,84kg,34,"['핏 좋습니다 ', <br/>, '살짝 끼는 핏인데 소재가 좋아서 늘어날거같습니다']",NaN,0,2,2,2,2,0,0,0,0
946853,~TOP~,2020.08.10,1168907,1,0,174cm,79kg,32,['상품 잘 받았습니다 배송도 빠르게 잘 받았습니다'],NaN,0,2,2,2,2,0,0,0,0
946854,똘똘똘,2020.08.07,1168907,1,0,183cm,90kg,36,['가성비좋은 무탠다드 감사합니다 번창하세요 옷 괜찮습니다 감사합니다'],NaN,0,2,2,2,2,0,0,0,0
946855,브론즈_910d82e626b,2020.08.05,1168907,1,0,175cm,76kg,32,"['32인데 허리랑 허벅지가 좀 타이트해요', <br/>, '길이는 적당히 깁니다']",NaN,0,2,2,2,1,0,0,0,0


In [7]:
df_pants.head()

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020.12.23,1281045,1,0,173cm,69kg,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0


In [8]:
df_pants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1504967 entries, 0 to 946856
Data columns (total 19 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   userName       1503404 non-null  object
 1   date           1504967 non-null  object
 2   goodsNo        1504967 non-null  int64 
 3   userSexMen     1504967 non-null  int64 
 4   userSexWomen   1504967 non-null  int64 
 5   userHeight     1504967 non-null  object
 6   userWeight     1504967 non-null  object
 7   goodsSize      1504967 non-null  object
 8   reviewContent  1504967 non-null  object
 9   reviewImg      434510 non-null   object
 10  reviewStyle    1504967 non-null  int64 
 11  size           1504967 non-null  int64 
 12  bright         1504967 non-null  int64 
 13  color          1504967 non-null  int64 
 14  thickness      1504967 non-null  int64 
 15  weightness     1504967 non-null  int64 
 16  touch          1504967 non-null  int64 
 17  helpNo         1504967 non-n

In [9]:
df_pants.shape

(1504967, 19)

### 탈퇴 인원 제거

In [10]:
df_pants.dropna(subset="userName", axis=0, inplace=True)
df_pants = df_pants.reset_index(drop=True)

In [11]:
df_pants.shape

(1503404, 19)

### Date를 년, 월, 일로 등분

In [12]:
df_pants.dropna(subset="date", axis=0, inplace=True)

In [13]:
df_pants.shape

(1503404, 19)

In [14]:
df_pants.insert(2, "dateDay", 0, True)
df_pants.insert(2, "dateMonth", 0, True)
df_pants.insert(2, "dateYear", 0, True)

In [15]:
df_pants.columns

Index(['userName', 'date', 'dateYear', 'dateMonth', 'dateDay', 'goodsNo',
       'userSexMen', 'userSexWomen', 'userHeight', 'userWeight', 'goodsSize',
       'reviewContent', 'reviewImg', 'reviewStyle', 'size', 'bright', 'color',
       'thickness', 'weightness', 'touch', 'helpNo', 'styleLikeNo'],
      dtype='object')

In [16]:
df_pants['dateYear'] = df_pants.apply(lambda x: x['date'].split('.')[0], axis=1)
df_pants['dateMonth'] = df_pants.apply(lambda x: x['date'].split('.')[1], axis=1)
df_pants['dateDay'] = df_pants.apply(lambda x: x['date'].split('.')[2], axis=1)

In [17]:
df_pants.head()

,userName,date,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022.01.10,2022,01,10,2258181,1,0,171cm,74kg,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021.12.24,2021,12,24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022.01.10,2022,01,10,2258181,1,0,171cm,74kg,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021.12.24,2021,12,24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020.12.23,2020,12,23,1281045,1,0,173cm,69kg,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0


In [18]:
df_pants = df_pants.drop('date', axis=1)

In [19]:
df_pants.dateYear = df_pants.dateYear.astype('int')
df_pants.dateMonth = df_pants.dateMonth.astype('int')
df_pants.dateDay = df_pants.dateDay.astype('int')

### goodsNo

In [20]:
df_pants.dropna(subset="goodsNo", axis=0, inplace=True)

In [21]:
df_pants.shape

(1503404, 21)

### 성별

In [22]:
df_pants[df_pants.userSexMen == 1].shape

(1289057, 21)

In [23]:
df_pants[df_pants.userSexWomen==1].shape

(214191, 21)

In [24]:
a = list((df_pants[df_pants.userSexMen==0][df_pants.userSexWomen==0]).index)
a

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_3804\1405742730.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = list((df_pants[df_pants.userSexMen==0][df_pants.userSexWomen==0]).index)


[7171,
 18542,
 25500,
 36751,
 47062,
 71306,
 81862,
 99514,
 99554,
 111520,
 113794,
 118877,
 130966,
 132652,
 138269,
 160093,
 164777,
 183899,
 205717,
 205720,
 220326,
 224275,
 242825,
 252431,
 252905,
 252942,
 261982,
 262476,
 263052,
 294324,
 327051,
 331165,
 347768,
 347822,
 390972,
 428956,
 429008,
 441682,
 459069,
 463433,
 510674,
 510696,
 512850,
 512872,
 523450,
 523565,
 552769,
 556256,
 561145,
 565234,
 579109,
 586481,
 598967,
 598975,
 600264,
 615534,
 639162,
 644467,
 660585,
 661990,
 667649,
 684220,
 685896,
 685904,
 690435,
 693422,
 707263,
 707273,
 707505,
 726545,
 726553,
 748025,
 748027,
 748789,
 750169,
 758905,
 758909,
 770338,
 771845,
 772430,
 775729,
 781779,
 804721,
 815861,
 832935,
 848221,
 848222,
 848277,
 851855,
 851880,
 851919,
 851945,
 872400,
 872430,
 872889,
 887390,
 892911,
 897508,
 911937,
 911955,
 924481,
 951355,
 962372,
 969276,
 1033484,
 1061233,
 1068316,
 1075208,
 1088355,
 1093537,
 1093538,
 110

In [25]:
df_pants = df_pants.drop(a)
df_pants.reset_index(drop=True)

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022,1,10,2258181,1,0,171cm,74kg,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021,12,24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022,1,10,2258181,1,0,171cm,74kg,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021,12,24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020,12,23,1281045,1,0,173cm,69kg,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503243,마산동호랭이,2020,8,10,1168907,1,0,178cm,84kg,34,"['핏 좋습니다 ', <br/>, '살짝 끼는 핏인데 소재가 좋아서 늘어날거같습니다']",NaN,0,2,2,2,2,0,0,0,0
1503244,~TOP~,2020,8,10,1168907,1,0,174cm,79kg,32,['상품 잘 받았습니다 배송도 빠르게 잘 받았습니다'],NaN,0,2,2,2,2,0,0,0,0
1503245,똘똘똘,2020,8,7,1168907,1,0,183cm,90kg,36,['가성비좋은 무탠다드 감사합니다 번창하세요 옷 괜찮습니다 감사합니다'],NaN,0,2,2,2,2,0,0,0,0
1503246,브론즈_910d82e626b,2020,8,5,1168907,1,0,175cm,76kg,32,"['32인데 허리랑 허벅지가 좀 타이트해요', <br/>, '길이는 적당히 깁니다']",NaN,0,2,2,2,1,0,0,0,0


In [26]:
1289057+214191

1503248

### 키와 몸무게

In [27]:
df_pants.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [28]:
df_pants.shape

(1503248, 21)

In [29]:
df_pants.userHeight = df_pants.userHeight.apply(lambda x: x.replace("cm", ""))
df_pants.userHeight = df_pants.userHeight.astype('int64')

In [30]:
df_pants.userWeight = df_pants.userWeight.apply(lambda x: x.replace("kg", ""))
df_pants.userWeight = df_pants.userWeight.astype('int64')

### 이미지

In [31]:
df_pants.reviewStyle.value_counts()

0    1069312
1     433936
Name: reviewStyle, dtype: int64

### 사이즈

In [32]:
sub = df_pants[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset="goodsNo", inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', 0, True)
sub['index'] = range(4701, 4701+len(sub))
sub.index = range(4701, 4701+len(sub))
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))

In [33]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [34]:
len(sub)

74138

In [35]:
for i in range(4701, len(sub)+4701):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
4701,2258181,M,2258181,4701
4702,2258181,L,2258181,4702
4703,1281045,S,1281045,4703
4704,2328543,L,2328543,4704
4705,2346636,블랙/XL,2346636,4705
...,...,...,...,...
78834,1168907,33,1168907,78834
78835,1168907,36,1168907,78835
78836,1168907,38,1168907,78836
78837,1168907,40,1168907,78837


In [36]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74138 entries, 4701 to 78838
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   goodsNo     74138 non-null  object
 1   goodsSize   74138 non-null  object
 2   newGoodsNo  74138 non-null  object
 3   index       74138 non-null  object
dtypes: object(4)
memory usage: 2.3+ MB


In [37]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub

,goodsNo,goodsSize,newGoodsNo,index
4701,2258181,M,22581814701,4701
4702,2258181,L,22581814702,4702
4703,1281045,S,12810454703,4703
4704,2328543,L,23285434704,4704
4705,2346636,블랙/XL,23466364705,4705
...,...,...,...,...
78834,1168907,33,116890778834,78834
78835,1168907,36,116890778835,78835
78836,1168907,38,116890778836,78836
78837,1168907,40,116890778837,78837


In [38]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

In [39]:
df_pants

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022,1,10,2258181,1,0,171,74,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021,12,24,2258181,1,0,175,80,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022,1,10,2258181,1,0,171,74,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021,12,24,2258181,1,0,175,80,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020,12,23,1281045,1,0,173,69,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503399,마산동호랭이,2020,8,10,1168907,1,0,178,84,34,"['핏 좋습니다 ', <br/>, '살짝 끼는 핏인데 소재가 좋아서 늘어날거같습니다']",NaN,0,2,2,2,2,0,0,0,0
1503400,~TOP~,2020,8,10,1168907,1,0,174,79,32,['상품 잘 받았습니다 배송도 빠르게 잘 받았습니다'],NaN,0,2,2,2,2,0,0,0,0
1503401,똘똘똘,2020,8,7,1168907,1,0,183,90,36,['가성비좋은 무탠다드 감사합니다 번창하세요 옷 괜찮습니다 감사합니다'],NaN,0,2,2,2,2,0,0,0,0
1503402,브론즈_910d82e626b,2020,8,5,1168907,1,0,175,76,32,"['32인데 허리랑 허벅지가 좀 타이트해요', <br/>, '길이는 적당히 깁니다']",NaN,0,2,2,2,1,0,0,0,0


#### month 당 사는 옷 수

In [40]:
df_pants.insert(5, 'month12', 0, True)
df_pants.insert(5, 'month11', 0, True)
df_pants.insert(5, 'month10', 0, True)
df_pants.insert(5, 'month9', 0, True)
df_pants.insert(5, 'month8', 0, True)
df_pants.insert(5, 'month7', 0, True)
df_pants.insert(5, 'month6', 0, True)
df_pants.insert(5, 'month5', 0, True)
df_pants.insert(5, 'month4', 0, True)
df_pants.insert(5, 'month3', 0, True)
df_pants.insert(5, 'month2', 0, True)
df_pants.insert(5, 'month1', 0, True)

In [41]:
df_pants.month1 = df_pants.apply(lambda x: 1 if x.dateMonth==1 else x.month1, axis=1)
df_pants.month2 = df_pants.apply(lambda x: 1 if x.dateMonth==2 else x.month2, axis=1)
df_pants.month3 = df_pants.apply(lambda x: 1 if x.dateMonth==3 else x.month3, axis=1)
df_pants.month4 = df_pants.apply(lambda x: 1 if x.dateMonth==4 else x.month4, axis=1)
df_pants.month5 = df_pants.apply(lambda x: 1 if x.dateMonth==5 else x.month5, axis=1)
df_pants.month6 = df_pants.apply(lambda x: 1 if x.dateMonth==6 else x.month6, axis=1)
df_pants.month7 = df_pants.apply(lambda x: 1 if x.dateMonth==7 else x.month7, axis=1)
df_pants.month8 = df_pants.apply(lambda x: 1 if x.dateMonth==8 else x.month8, axis=1)
df_pants.month9 = df_pants.apply(lambda x: 1 if x.dateMonth==9 else x.month9, axis=1)
df_pants.month10 = df_pants.apply(lambda x: 1 if x.dateMonth==10 else x.month10, axis=1)
df_pants.month11 = df_pants.apply(lambda x: 1 if x.dateMonth==11 else x.month11, axis=1)
df_pants.month12 = df_pants.apply(lambda x: 1 if x.dateMonth==12 else x.month12, axis=1)

In [42]:
d = df_pants.groupby(['goodsNo', 'goodsSize']).sum()[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]
d

month1  month2  month3  month4  month5  month6  month7  \
goodsNo goodsSize                                                           
25868   28/28           0       0       1       1       0       0       0   
        28/30           0       0       0       0       0       0       1   
        28/32           0       0       0       0       0       0       0   
        29/28           0       0       2       0       1       0       1   
        29/30           0       0       0       0       0       0       0   
...                   ...     ...     ...     ...     ...     ...     ...   
2407517 30              0       0       1       0       0       0       0   
2407719 L               0       0       1       0       0       0       0   
2410780 L               0       0       1       0       0       0       0   
2411470 그레이/L(75)       0       0       1       0       0       0       0   
2413456 M(85)           0       0       2       0       0       0       0   

                   month8  month9  month10  month11  month12  
goodsNo goodsSize                                             
25868   28/28           0       1        2        1        0  
        28/30           0       0        0        0        0  
        28/32           0       1        0        0        0  
        29/28           0       1        2        1        1  
        29/30           1       0        1        0        0  
...                   ...     ...      ...      ...      ...  
2407517 30              0       0        0        0        0  
2407719 L               0       0        0        0        0  
2410780 L               0       0        0        0        0  
2411470 그레이/L(75)       0       0        0        0        0  
2413456 M(85)           0       0        0        0        0  

[74138 rows x 12 columns]

#### 키, 몸무게 평균 구하기

In [43]:
c = df_pants.groupby(['goodsNo', 'goodsSize'], as_index=False).mean()[['goodsNo', 'goodsSize', 'userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]
c

,goodsNo,goodsSize,userHeight,userWeight,userSexMen,userSexWomen
0,25868,28/28,167.833333,54.333333,0.666667,0.333333
1,25868,28/30,174.000000,53.000000,1.000000,0.000000
2,25868,28/32,167.000000,50.000000,0.000000,1.000000
3,25868,29/28,170.222222,63.444444,1.000000,0.000000
4,25868,29/30,174.500000,62.500000,1.000000,0.000000
...,...,...,...,...,...,...
74133,2407517,30,172.000000,65.000000,1.000000,0.000000
74134,2407719,L,183.000000,78.000000,1.000000,0.000000
74135,2410780,L,169.000000,56.000000,0.000000,1.000000
74136,2411470,그레이/L(75),167.000000,59.000000,0.000000,1.000000


In [44]:
sub.sort_values(['goodsNo', 'goodsSize'], inplace=True)
c.sort_values(['goodsNo', 'goodsSize'], inplace=True)
d.sort_values(['goodsNo', 'goodsSize'], inplace=True)
sub.index = range(len(d))
sub

,goodsNo,goodsSize,newGoodsNo,index
0,25868,28/28,258680052216,52216
1,25868,28/30,258680052219,52219
2,25868,28/32,258680052205,52205
3,25868,29/28,258680052206,52206
4,25868,29/30,258680052202,52202
...,...,...,...,...
74133,2407517,30,240751752981,52981
74134,2407719,L,240771913225,13225
74135,2410780,L,241078053058,53058
74136,2411470,그레이/L(75),241147046648,46648


In [45]:
d.index = range(len(d))
d

,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,0,0,1,1,0,0,0,0,1,2,1,0
1,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,2,0,1,0,1,0,1,2,1,1
4,0,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
74133,0,0,1,0,0,0,0,0,0,0,0,0
74134,0,0,1,0,0,0,0,0,0,0,0,0
74135,0,0,1,0,0,0,0,0,0,0,0,0
74136,0,0,1,0,0,0,0,0,0,0,0,0


In [46]:
sub = pd.concat([sub, c[['userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]], axis=1)
sub = pd.concat([sub, d[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]], axis=1)
sub.index = range(len(sub))
sub

,goodsNo,goodsSize,newGoodsNo,index,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,25868,28/28,258680052216,52216,167.833333,54.333333,0.666667,0.333333,0,0,1,1,0,0,0,0,1,2,1,0
1,25868,28/30,258680052219,52219,174.000000,53.000000,1.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0
2,25868,28/32,258680052205,52205,167.000000,50.000000,0.000000,1.000000,0,0,0,0,0,0,0,0,1,0,0,0
3,25868,29/28,258680052206,52206,170.222222,63.444444,1.000000,0.000000,0,0,2,0,1,0,1,0,1,2,1,1
4,25868,29/30,258680052202,52202,174.500000,62.500000,1.000000,0.000000,0,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74133,2407517,30,240751752981,52981,172.000000,65.000000,1.000000,0.000000,0,0,1,0,0,0,0,0,0,0,0,0
74134,2407719,L,240771913225,13225,183.000000,78.000000,1.000000,0.000000,0,0,1,0,0,0,0,0,0,0,0,0
74135,2410780,L,241078053058,53058,169.000000,56.000000,0.000000,1.000000,0,0,1,0,0,0,0,0,0,0,0,0
74136,2411470,그레이/L(75),241147046648,46648,167.000000,59.000000,0.000000,1.000000,0,0,1,0,0,0,0,0,0,0,0,0


In [47]:
sub.head()

,goodsNo,goodsSize,newGoodsNo,index,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,25868,28/28,258680052216,52216,167.833333,54.333333,0.666667,0.333333,0,0,1,1,0,0,0,0,1,2,1,0
1,25868,28/30,258680052219,52219,174.000000,53.000000,1.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0
2,25868,28/32,258680052205,52205,167.000000,50.000000,0.000000,1.000000,0,0,0,0,0,0,0,0,1,0,0,0
3,25868,29/28,258680052206,52206,170.222222,63.444444,1.000000,0.000000,0,0,2,0,1,0,1,0,1,2,1,1
4,25868,29/30,258680052202,52202,174.500000,62.500000,1.000000,0.000000,0,0,0,0,0,0,0,1,0,1,0,0


In [48]:
df_pants = pd.merge(left=df_pants, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_pants.head()

,userName,dateYear,dateMonth,dateDay,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,기무태,2022,1,10,2258181,1,0,0,0,0,0,0,0,0,0,0,0,1,0,171,74,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0,22581814701
1,10407,2021,12,24,2258181,0,0,0,0,0,0,0,0,0,0,0,1,1,0,175,80,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0,22581814702
2,기무태,2022,1,10,2258181,1,0,0,0,0,0,0,0,0,0,0,0,1,0,171,74,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0,22581814701
3,10407,2021,12,24,2258181,0,0,0,0,0,0,0,0,0,0,0,1,1,0,175,80,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0,22581814702
4,브론즈_f0b04c5803c,2020,12,23,1281045,0,0,0,0,0,0,0,0,0,0,0,1,1,0,173,69,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0,12810454703


In [159]:
len(sub['goodsNo'].unique())

1800

In [160]:
sub.shape

(2654, 20)

In [188]:
sub.to_csv('size_pants.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

### Contents

In [101]:
print(df_pants.reviewContent[0])
print(df_pants.reviewContent[1].split(","))

['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 사이즈 구애받지 않고 다 입으실 수 있을거 같아요! 허리끈이 따로 있어서 다양하게 스타일링 할 수 있어 좋네요!']
["['여름에 편하게 입기 좋을것 같아요!'", ' <br/>', " '생각보다 많이 파여서 조심해야ㅎㅐ요ㅠㅠ'", ' <br/>', " '앞뒤가 언발란스한 기장때문에 심심하지 않아 좋네요!']"]


In [102]:
# span 태그가 들어가있는 리뷰내용 전처리

cnt = 0
result = []
for i in df_pants.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

0


In [103]:
df_pants.reviewContent = df_pants.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("'", "").replace(", <br/>", "").split(","))

In [104]:
print(df_pants.reviewContent[0])
print(df_pants.reviewContent[1])

['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 사이즈 구애받지 않고 다 입으실 수 있을거 같아요! 허리끈이 따로 있어서 다양하게 스타일링 할 수 있어 좋네요!']
['여름에 편하게 입기 좋을것 같아요!', '생각보다 많이 파여서 조심해야ㅎㅐ요ㅠㅠ', '앞뒤가 언발란스한 기장때문에 심심하지 않아 좋네요!']


### 그 외

In [105]:
df_pants['size'].value_counts()

2    6916
1    1103
3     428
0       1
Name: size, dtype: int64

In [106]:
df_pants.bright.value_counts()

2    6711
1     934
3     802
0       1
Name: bright, dtype: int64

In [107]:
df_pants.color.value_counts()

2    5667
1    2646
3     134
0       1
Name: color, dtype: int64

In [108]:
df_pants.thickness.value_counts()

0    8446
3       2
Name: thickness, dtype: int64

In [109]:
df_pants.weightness.value_counts()

0    8448
Name: weightness, dtype: int64

In [110]:
df_pants.touch.value_counts()

0    8448
Name: touch, dtype: int64

### helpNo, styleLikeNo는 모두 0이다. 제거 가능성 높아짐

In [111]:
df_pants.describe()

,dateYear,dateMonth,dateDay,newClothNo,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
count,8448.000000,8448.000000,8448.000000,8.448000e+03,8.448000e+03,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.0,8448.0,8448.0,8448.0
mean,2021.150331,6.627012,15.122514,9.490916e+09,1.813087e+06,0.079664,0.104167,0.081084,0.035985,0.059067,0.077770,0.109730,0.122633,0.089252,0.088778,0.072798,0.079072,0.017045,0.982955,162.436553,52.915601,0.250473,1.919863,1.984138,1.702415,0.000710,0.0,0.0,0.0,0.0
std,0.503225,3.419035,8.839225,9.804172e+09,3.012917e+05,0.270788,0.305495,0.272981,0.186263,0.235764,0.267825,0.312572,0.328034,0.285124,0.284440,0.259821,0.269867,0.129448,0.129448,5.925707,12.765438,0.433311,0.418687,0.453584,0.490935,0.046157,0.0,0.0,0.0,0.0
min,2020.000000,1.000000,1.000000,1.600404e+07,2.308530e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2021.000000,3.000000,7.000000,1.871714e+09,1.609158e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,159.000000,48.000000,0.000000,2.000000,2.000000,1.000000,0.000000,0.0,0.0,0.0,0.0
50%,2021.000000,7.000000,15.000000,2.153849e+09,1.877330e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,162.000000,52.000000,0.000000,2.000000,2.000000,2.000000,0.000000,0.0,0.0,0.0,0.0
75%,2021.000000,9.000000,23.000000,1.875664e+10,2.009663e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,166.000000,56.000000,1.000000,2.000000,2.000000,2.000000,0.000000,0.0,0.0,0.0,0.0
max,2022.000000,12.000000,31.000000,9.930520e+10,2.397414e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,200.000000,1000.000000,1.000000,3.000000,3.000000,3.000000,3.000000,0.0,0.0,0.0,0.0


# Outer 리뷰 전처리

In [204]:
df_outer1 = pd.read_csv('./musinsa_review_outer_1.csv')
df_outer2 = pd.read_csv('./musinsa_review_outer_2.csv')

In [205]:
print(df_outer1.shape, df_outer2.shape)

(558110, 19) (946857, 19)


In [206]:
df_outer = pd.concat([df_outer1, df_outer2], axis=0)
df_outer.shape

(1504967, 19)

In [207]:
df_outer

,userName,date,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022.01.10,2258181,1,0,171cm,74kg,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021.12.24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020.12.23,1281045,1,0,173cm,69kg,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946852,마산동호랭이,2020.08.10,1168907,1,0,178cm,84kg,34,"['핏 좋습니다 ', <br/>, '살짝 끼는 핏인데 소재가 좋아서 늘어날거같습니다']",NaN,0,2,2,2,2,0,0,0,0
946853,~TOP~,2020.08.10,1168907,1,0,174cm,79kg,32,['상품 잘 받았습니다 배송도 빠르게 잘 받았습니다'],NaN,0,2,2,2,2,0,0,0,0
946854,똘똘똘,2020.08.07,1168907,1,0,183cm,90kg,36,['가성비좋은 무탠다드 감사합니다 번창하세요 옷 괜찮습니다 감사합니다'],NaN,0,2,2,2,2,0,0,0,0
946855,브론즈_910d82e626b,2020.08.05,1168907,1,0,175cm,76kg,32,"['32인데 허리랑 허벅지가 좀 타이트해요', <br/>, '길이는 적당히 깁니다']",NaN,0,2,2,2,1,0,0,0,0


In [209]:
df_outer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1504967 entries, 0 to 946856
Data columns (total 19 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   userName       1503404 non-null  object
 1   date           1504967 non-null  object
 2   goodsNo        1504967 non-null  int64 
 3   userSexMen     1504967 non-null  int64 
 4   userSexWomen   1504967 non-null  int64 
 5   userHeight     1504967 non-null  object
 6   userWeight     1504967 non-null  object
 7   goodsSize      1504967 non-null  object
 8   reviewContent  1504967 non-null  object
 9   reviewImg      434510 non-null   object
 10  reviewStyle    1504967 non-null  int64 
 11  size           1504967 non-null  int64 
 12  bright         1504967 non-null  int64 
 13  color          1504967 non-null  int64 
 14  thickness      1504967 non-null  int64 
 15  weightness     1504967 non-null  int64 
 16  touch          1504967 non-null  int64 
 17  helpNo         1504967 non-n

In [210]:
df_outer.shape

(1504967, 19)

### 탈퇴 인원 제거

In [211]:
df_outer.dropna(subset="userName", axis=0, inplace=True)
df_outer = df_outer.reset_index(drop=True)

In [212]:
df_outer.shape

(1503404, 19)

### Date를 년, 월, 일로 등분

In [213]:
df_outer.dropna(subset="date", axis=0, inplace=True)

In [214]:
df_outer.shape

(1503404, 19)

In [215]:
df_outer.insert(2, "dateDay", 0, True)
df_outer.insert(2, "dateMonth", 0, True)
df_outer.insert(2, "dateYear", 0, True)

In [216]:
df_outer.columns

Index(['userName', 'date', 'dateYear', 'dateMonth', 'dateDay', 'goodsNo',
       'userSexMen', 'userSexWomen', 'userHeight', 'userWeight', 'goodsSize',
       'reviewContent', 'reviewImg', 'reviewStyle', 'size', 'bright', 'color',
       'thickness', 'weightness', 'touch', 'helpNo', 'styleLikeNo'],
      dtype='object')

In [217]:
df_outer['dateYear'] = df_outer.apply(lambda x: x['date'].split('.')[0], axis=1)
df_outer['dateMonth'] = df_outer.apply(lambda x: x['date'].split('.')[1], axis=1)
df_outer['dateDay'] = df_outer.apply(lambda x: x['date'].split('.')[2], axis=1)

In [218]:
df_outer.head()

,userName,date,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022.01.10,2022,01,10,2258181,1,0,171cm,74kg,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021.12.24,2021,12,24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022.01.10,2022,01,10,2258181,1,0,171cm,74kg,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021.12.24,2021,12,24,2258181,1,0,175cm,80kg,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020.12.23,2020,12,23,1281045,1,0,173cm,69kg,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0


In [219]:
df_outer = df_outer.drop('date', axis=1)

In [220]:
df_outer.dateYear = df_outer.dateYear.astype('int')
df_outer.dateMonth = df_outer.dateMonth.astype('int')
df_outer.dateDay = df_outer.dateDay.astype('int')

### goodsNo

In [221]:
df_outer.dropna(subset="goodsNo", axis=0, inplace=True)

In [222]:
df_outer.shape

(1503404, 21)

### 성별

In [223]:
df_outer[df_outer.userSexMen == 1].shape

(1289057, 21)

In [224]:
df_outer[df_outer.userSexWomen==1].shape

(214191, 21)

In [225]:
df_outer[df_outer.userSexMen==0][df_outer.userSexWomen==0]

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_588\1997673721.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_pants[df_pants.userSexMen==0][df_pants.userSexWomen==0]


,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
7171,박종우기,2020,7,22,1310218,0,0,0cm,0kg,BASIC/32,"['밑단 쪼이는 거 정말 잘 쪼여지네요 그리고', <br/>, '편하게 입는 느낌 ...",https://image.msscdn.net/data/estimate/1310218...,1,2,1,1,2,0,0,0,0
18542,치로미,2020,6,9,1003446,0,0,0cm,0kg,1,['Lmc 바지는 처음인데 생각보다 뭔가 디자인이 이상하네요 통이 너무 크고 통만큼...,https://image.msscdn.net/data/estimate/1003446...,1,1,2,2,2,0,0,0,0
25500,경기도수원시,2020,7,30,1396557,0,0,0cm,0kg,30,['상품의 핏이나 원단이 너무 맘에 들어요 다음에 또 구매하고 싶어요!'],https://image.msscdn.net/data/estimate/1396557...,1,2,2,2,2,0,0,0,0
36751,김겅겅,2020,6,23,1418892,0,0,0cm,0kg,M,['코튼함유량이 많아서 생각보다 얇지는 않습니다 그래도 일반 면바지보단 시원하네요 ...,https://image.msscdn.net/data/estimate/1418892...,1,2,2,2,2,0,0,0,0
47062,뿁숍러,2020,7,11,1343575,0,0,0cm,0kg,34,['허리가 고무줄 안 하면 큰편입니다!! 보통보다 하나 작게 주문하셔도 될 것 같아요'],https://image.msscdn.net/data/estimate/1343575...,1,1,2,2,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490675,낭만베짱이,2020,3,6,545333,0,0,178cm,80kg,48(32~33)/cutting,['처음 구매하는 거라 사이즈가 어떨까 궁금했는데 평소 32 입어서 기본으로 구매했...,https://image.msscdn.net/data/estimate/545333_...,1,2,3,1,1,0,0,0,0
1494639,이든킴2,2020,6,19,1077204,0,0,0cm,0kg,M,['넘 이쁘고 소재도 겁나 얇고 시원해서 좋네요 한여름에도 착용 할 수 있을 거 같...,https://image.msscdn.net/data/estimate/1077204...,1,2,2,2,3,0,0,0,0
1494645,고고고고은별,2020,5,27,1077204,0,0,0cm,0kg,S,['소재가 시원해서 여름에 입기 좋아요!! 클까봐 걱정했는데 s사이즈 사니까 딱 맞...,https://image.msscdn.net/data/estimate/1077204...,1,2,2,2,3,0,0,0,0
1498757,닉다섯임,2020,6,23,825757,0,0,0cm,0kg,블랙(S)/홀더구매안함,['따릉이릉 타고 따릉따릉 달려보아여 신난당 히히'],https://image.msscdn.net/data/estimate/825757_...,1,2,2,2,2,0,0,0,0


In [226]:
1289057+214191

1503248

### 키와 몸무게

In [227]:
df_outer.dropna(subset=["userHeight", "userWeight"], axis=0, inplace=True)

In [228]:
df_outer.shape

(1503404, 21)

In [229]:
df_outer.userHeight = df_outer.userHeight.apply(lambda x: x.replace("cm", ""))
df_outer.userHeight = df_outer.userHeight.astype('int64')

In [230]:
df_outer.userWeight = df_outer.userWeight.apply(lambda x: x.replace("kg", ""))
df_outer.userWeight = df_outer.userWeight.astype('int64')

### 이미지

In [231]:
df_outer.reviewStyle.value_counts()

0    1069313
1     434091
Name: reviewStyle, dtype: int64

### 사이즈

In [250]:
sub = df_outer[["goodsNo", "goodsSize"]]
sub = sub.drop_duplicates(subset = ["goodsNo", "goodsSize"]).reset_index(drop=True)
sub.dropna(subset="goodsNo", inplace=True)
sub.insert(2, 'newGoodsNo', sub.goodsNo, True)
sub.insert(3, 'index', 0, True)
sub['index'] = range(78841, 78841+len(sub))
sub.index = range(78841, 78841+len(sub))
# sub.newGoodsNo = sub.newGoodsNo.apply(lambda x: round(x))

In [251]:
sub.goodsNo = sub.goodsNo.astype('str')
sub['index'] = sub['index'].astype('str')
sub.newGoodsNo = sub.newGoodsNo.astype('str')

In [252]:
len(sub)

74140

In [253]:
for i in range(78841, len(sub)+78841):
    if len(sub.newGoodsNo[i]) < 7:
        while len(sub.newGoodsNo[i]) < 7:
            sub.newGoodsNo[i] += '0'
sub

,goodsNo,goodsSize,newGoodsNo,index
4701,2258181,M,2258181,4701
4702,2258181,L,2258181,4702
4703,1281045,S,1281045,4703
4704,2328543,L,2328543,4704
4705,2346636,블랙/XL,2346636,4705
...,...,...,...,...
78836,1168907,33,1168907,78836
78837,1168907,36,1168907,78837
78838,1168907,38,1168907,78838
78839,1168907,40,1168907,78839


In [254]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74140 entries, 4701 to 78840
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   goodsNo     74140 non-null  object
 1   goodsSize   74140 non-null  object
 2   newGoodsNo  74140 non-null  object
 3   index       74140 non-null  object
dtypes: object(4)
memory usage: 2.3+ MB


In [255]:
sub['newGoodsNo'] = sub.apply(lambda x: x['newGoodsNo']+x['index'], axis=1)
sub

,goodsNo,goodsSize,newGoodsNo,index
4701,2258181,M,22581814701,4701
4702,2258181,L,22581814702,4702
4703,1281045,S,12810454703,4703
4704,2328543,L,23285434704,4704
4705,2346636,블랙/XL,23466364705,4705
...,...,...,...,...
78836,1168907,33,116890778836,78836
78837,1168907,36,116890778837,78837
78838,1168907,38,116890778838,78838
78839,1168907,40,116890778839,78839


In [256]:
sub.goodsNo = sub.goodsNo.astype('int')
sub.newGoodsNo = sub.newGoodsNo.astype('int64')

In [258]:
df_outer

,userName,dateYear,dateMonth,dateDay,goodsNo,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
0,기무태,2022,1,10,2258181,1,0,171,74,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
1,10407,2021,12,24,2258181,1,0,175,80,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0
2,기무태,2022,1,10,2258181,1,0,171,74,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0
3,10407,2021,12,24,2258181,1,0,175,80,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0
4,브론즈_f0b04c5803c,2020,12,23,1281045,1,0,173,69,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503399,마산동호랭이,2020,8,10,1168907,1,0,178,84,34,"['핏 좋습니다 ', <br/>, '살짝 끼는 핏인데 소재가 좋아서 늘어날거같습니다']",NaN,0,2,2,2,2,0,0,0,0
1503400,~TOP~,2020,8,10,1168907,1,0,174,79,32,['상품 잘 받았습니다 배송도 빠르게 잘 받았습니다'],NaN,0,2,2,2,2,0,0,0,0
1503401,똘똘똘,2020,8,7,1168907,1,0,183,90,36,['가성비좋은 무탠다드 감사합니다 번창하세요 옷 괜찮습니다 감사합니다'],NaN,0,2,2,2,2,0,0,0,0
1503402,브론즈_910d82e626b,2020,8,5,1168907,1,0,175,76,32,"['32인데 허리랑 허벅지가 좀 타이트해요', <br/>, '길이는 적당히 깁니다']",NaN,0,2,2,2,1,0,0,0,0


#### month 당 사는 옷 수

In [259]:
df_outer.insert(5, 'month12', 0, True)
df_outer.insert(5, 'month11', 0, True)
df_outer.insert(5, 'month10', 0, True)
df_outer.insert(5, 'month9', 0, True)
df_outer.insert(5, 'month8', 0, True)
df_outer.insert(5, 'month7', 0, True)
df_outer.insert(5, 'month6', 0, True)
df_outer.insert(5, 'month5', 0, True)
df_outer.insert(5, 'month4', 0, True)
df_outer.insert(5, 'month3', 0, True)
df_outer.insert(5, 'month2', 0, True)
df_outer.insert(5, 'month1', 0, True)

In [260]:
df_outer.month1 = df_outer.apply(lambda x: 1 if x.dateMonth==1 else x.month1, axis=1)
df_outer.month2 = df_outer.apply(lambda x: 1 if x.dateMonth==2 else x.month2, axis=1)
df_outer.month3 = df_outer.apply(lambda x: 1 if x.dateMonth==3 else x.month3, axis=1)
df_outer.month4 = df_outer.apply(lambda x: 1 if x.dateMonth==4 else x.month4, axis=1)
df_outer.month5 = df_outer.apply(lambda x: 1 if x.dateMonth==5 else x.month5, axis=1)
df_outer.month6 = df_outer.apply(lambda x: 1 if x.dateMonth==6 else x.month6, axis=1)
df_outer.month7 = df_outer.apply(lambda x: 1 if x.dateMonth==7 else x.month7, axis=1)
df_outer.month8 = df_outer.apply(lambda x: 1 if x.dateMonth==8 else x.month8, axis=1)
df_outer.month9 = df_outer.apply(lambda x: 1 if x.dateMonth==9 else x.month9, axis=1)
df_outer.month10 = df_outer.apply(lambda x: 1 if x.dateMonth==10 else x.month10, axis=1)
df_outer.month11 = df_outer.apply(lambda x: 1 if x.dateMonth==11 else x.month11, axis=1)
df_outer.month12 = df_outer.apply(lambda x: 1 if x.dateMonth==12 else x.month12, axis=1)

In [93]:
d = df_outer.groupby(['goodsNo', 'goodsSize']).sum()[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]
d

,,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
goodsNo,goodsSize,,,,,,,,,,,,
230853,none,0,0,0,0,0,0,1,0,0,0,0,0
387218,그레이헤더/S,0,0,0,0,0,0,0,1,0,0,0,0
435992,NONE,0,0,0,1,0,0,0,0,0,0,0,0
653344,none,0,0,0,0,0,0,0,0,0,0,1,0
677397,FREE,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386153,M,0,0,1,0,0,0,0,0,0,0,0,0
2396014,M,0,0,1,0,0,0,0,0,0,0,0,0
2396257,FREE,0,0,2,0,0,0,0,0,0,0,0,0


#### 키, 몸무게 평균 구하기

In [94]:
c = df_outer.groupby(['goodsNo', 'goodsSize'], as_index=False).mean()[['goodsNo', 'goodsSize', 'userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]
c

,goodsNo,goodsSize,userHeight,userWeight,userSexMen,userSexWomen
0,230853,none,155.0,45.0,0.0,1.0
1,387218,그레이헤더/S,161.0,43.0,0.0,1.0
2,435992,NONE,162.0,55.0,0.0,1.0
3,653344,none,155.0,55.0,0.0,1.0
4,677397,FREE,163.0,55.0,0.0,1.0
...,...,...,...,...,...,...
2042,2386153,M,171.0,58.0,0.0,1.0
2043,2396014,M,173.0,53.0,0.0,1.0
2044,2396257,FREE,158.0,48.0,0.0,1.0
2045,2396259,FREE,158.0,48.0,0.0,1.0


In [95]:
sub.sort_values(['goodsNo', 'goodsSize'], inplace=True)
c.sort_values(['goodsNo', 'goodsSize'], inplace=True)
d.sort_values(['goodsNo', 'goodsSize'], inplace=True)
sub.index = range(len(d))

,goodsNo,goodsSize,newGoodsNo,index
1368,230853,none,23085301368,1368
580,387218,그레이헤더/S,3872180580,580
549,435992,NONE,4359920549,549
1529,653344,none,65334401529,1529
411,677397,FREE,6773970411,411
...,...,...,...,...
867,2386153,M,2386153867,867
674,2396014,M,2396014674,674
1068,2396257,FREE,23962571068,1068
667,2396259,FREE,2396259667,667


In [96]:
d.index = range(len(d))
d

,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2042,0,0,1,0,0,0,0,0,0,0,0,0
2043,0,0,1,0,0,0,0,0,0,0,0,0
2044,0,0,2,0,0,0,0,0,0,0,0,0
2045,0,0,2,0,0,0,0,0,0,0,0,0


In [97]:
sub = pd.concat([sub, c[['userHeight', 'userWeight', 'userSexMen', 'userSexWomen']]], axis=1)
sub = pd.concat([sub, d[['month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']]], axis=1)
sub.index = range(len(sub))
sub

,goodsNo,goodsSize,newGoodsNo,index,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,230853,none,23085301368,1368,166.000000,65.000000,0.0,1.0,0,0,0,0,0,0,1,0,0,0,0,0
1,387218,그레이헤더/S,3872180580,580,158.000000,52.000000,0.0,1.0,0,0,0,0,0,0,0,1,0,0,0,0
2,435992,NONE,4359920549,549,168.000000,66.000000,0.0,1.0,0,0,0,0,0,0,0,0,0,1,0,0
3,653344,none,65334401529,1529,162.000000,52.500000,0.0,1.0,0,1,0,0,0,0,0,0,0,0,1,0
4,677397,FREE,6773970411,411,164.333333,62.500000,0.0,1.0,0,0,0,2,1,3,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2042,2386153,M,2386153867,867,159.257143,46.885714,0.0,1.0,0,4,1,5,4,0,6,9,3,2,1,0
2043,2396014,M,2396014674,674,170.000000,60.000000,0.0,1.0,0,0,0,0,1,0,0,0,0,0,0,0
2044,2396257,FREE,23962571068,1068,160.333333,52.666667,0.0,1.0,0,0,0,0,0,1,2,0,0,0,0,0
2045,2396259,FREE,2396259667,667,166.000000,51.000000,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0


In [48]:
df_outer = pd.merge(left=df_outer, right=sub[['goodsNo', 'goodsSize','newGoodsNo']], how="left", on=['goodsNo', 'goodsSize'])
df_outer.head()

,userName,dateYear,dateMonth,dateDay,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,goodsSize,reviewContent,reviewImg,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo,newGoodsNo
0,기무태,2022,1,10,2258181,1,0,0,0,0,0,0,0,0,0,0,0,1,0,171,74,M,"['세트별로 다사서 이번엔 바지만 샀는데', <br/>, '이것도 위에 같이 살걸 ...",https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0,22581814701
1,10407,2021,12,24,2258181,0,0,0,0,0,0,0,0,0,0,0,1,1,0,175,80,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],https://image.msscdn.net/data/estimate/2258181...,1,2,2,2,2,0,0,0,0,22581814702
2,기무태,2022,1,10,2258181,1,0,0,0,0,0,0,0,0,0,0,0,1,0,171,74,M,"['진짜 너무 이뻐서바지만 또사요', <br/>, '정말 최고 아이템']",NaN,0,2,2,2,2,0,0,0,0,22581814701
3,10407,2021,12,24,2258181,0,0,0,0,0,0,0,0,0,0,0,1,1,0,175,80,L,['배송도 빨랐고 사이즈도 딱이구 기모라서 따뜻합니댱'],NaN,0,2,2,2,2,0,0,0,0,22581814702
4,브론즈_f0b04c5803c,2020,12,23,1281045,0,0,0,0,0,0,0,0,0,0,0,1,1,0,173,69,S,"['제 값 하는 옷이에요, 핏도 너무 이쁘고 스판이라 너무 좋습니다. 배송 왔을 때...",https://image.msscdn.net/data/estimate/1281045...,1,2,2,2,2,0,0,0,0,12810454703


In [159]:
len(sub['goodsNo'].unique())

1800

In [160]:
sub.shape

(2654, 20)

In [188]:
sub.to_csv('size_outer.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

### Contents

In [101]:
print(df_outer.reviewContent[0])
print(df_outer.reviewContent[1].split(","))

['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 사이즈 구애받지 않고 다 입으실 수 있을거 같아요! 허리끈이 따로 있어서 다양하게 스타일링 할 수 있어 좋네요!']
["['여름에 편하게 입기 좋을것 같아요!'", ' <br/>', " '생각보다 많이 파여서 조심해야ㅎㅐ요ㅠㅠ'", ' <br/>', " '앞뒤가 언발란스한 기장때문에 심심하지 않아 좋네요!']"]


In [102]:
# span 태그가 들어가있는 리뷰내용 전처리

cnt = 0
result = []
for i in df_outer.reviewContent:
    if '<span>' in i:
        cnt += 1
        result.append(i)
print(cnt)

0


In [103]:
df_outer.reviewContent = df_outer.reviewContent.apply(lambda x: x.replace("[", "").replace("]", "").replace(" '", "").replace("'", "").replace(", <br/>", "").split(","))

In [104]:
print(df_outer.reviewContent[0])
print(df_outer.reviewContent[1])

['다크그레이 색상이라고 했는데 받아보면 진한 카키에 가깝습니다! 품이 매우 커서 사이즈 구애받지 않고 다 입으실 수 있을거 같아요! 허리끈이 따로 있어서 다양하게 스타일링 할 수 있어 좋네요!']
['여름에 편하게 입기 좋을것 같아요!', '생각보다 많이 파여서 조심해야ㅎㅐ요ㅠㅠ', '앞뒤가 언발란스한 기장때문에 심심하지 않아 좋네요!']


### 그 외

In [105]:
df_outer['size'].value_counts()

2    6916
1    1103
3     428
0       1
Name: size, dtype: int64

In [106]:
df_outer.bright.value_counts()

2    6711
1     934
3     802
0       1
Name: bright, dtype: int64

In [107]:
df_outer.color.value_counts()

2    5667
1    2646
3     134
0       1
Name: color, dtype: int64

In [108]:
df_outer.thickness.value_counts()

0    8446
3       2
Name: thickness, dtype: int64

In [109]:
df_outer.weightness.value_counts()

0    8448
Name: weightness, dtype: int64

In [110]:
df_outer.touch.value_counts()

0    8448
Name: touch, dtype: int64

### helpNo, styleLikeNo는 모두 0이다. 제거 가능성 높아짐

In [111]:
df_outer.describe()

,dateYear,dateMonth,dateDay,newClothNo,goodsNo,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,userSexMen,userSexWomen,userHeight,userWeight,reviewStyle,size,bright,color,thickness,weightness,touch,helpNo,styleLikeNo
count,8448.000000,8448.000000,8448.000000,8.448000e+03,8.448000e+03,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.000000,8448.0,8448.0,8448.0,8448.0
mean,2021.150331,6.627012,15.122514,9.490916e+09,1.813087e+06,0.079664,0.104167,0.081084,0.035985,0.059067,0.077770,0.109730,0.122633,0.089252,0.088778,0.072798,0.079072,0.017045,0.982955,162.436553,52.915601,0.250473,1.919863,1.984138,1.702415,0.000710,0.0,0.0,0.0,0.0
std,0.503225,3.419035,8.839225,9.804172e+09,3.012917e+05,0.270788,0.305495,0.272981,0.186263,0.235764,0.267825,0.312572,0.328034,0.285124,0.284440,0.259821,0.269867,0.129448,0.129448,5.925707,12.765438,0.433311,0.418687,0.453584,0.490935,0.046157,0.0,0.0,0.0,0.0
min,2020.000000,1.000000,1.000000,1.600404e+07,2.308530e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2021.000000,3.000000,7.000000,1.871714e+09,1.609158e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,159.000000,48.000000,0.000000,2.000000,2.000000,1.000000,0.000000,0.0,0.0,0.0,0.0
50%,2021.000000,7.000000,15.000000,2.153849e+09,1.877330e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,162.000000,52.000000,0.000000,2.000000,2.000000,2.000000,0.000000,0.0,0.0,0.0,0.0
75%,2021.000000,9.000000,23.000000,1.875664e+10,2.009663e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,166.000000,56.000000,1.000000,2.000000,2.000000,2.000000,0.000000,0.0,0.0,0.0,0.0
max,2022.000000,12.000000,31.000000,9.930520e+10,2.397414e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,200.000000,1000.000000,1.000000,3.000000,3.000000,3.000000,3.000000,0.0,0.0,0.0,0.0
